In [89]:
import re
import random
import copy
import math

# Adjacent Vacant Status

In [90]:
# Find the adjacency of each cell in the grid
def calc_adj(grid):
    M, N = len(grid), len(grid[0])
    adj_mat = copy.deepcopy(grid)

    for i in range(M):
        for j in range(N):
            if grid[i][j] != '':
                adj_mat[i][j] = -1
            else:
                c = 0
                for dx, dy in [(0, -1), (-1, 0), (0, 1), (1, 0)]:
                    x, y = i, j
                    x += dx
                    y += dy
                    if 0 <= x < M and 0 <= y < N and grid[x][y] == '':
                        c += 1
                adj_mat[i][j] = c

    return adj_mat

# if a qubit has a vacant neighbor, returns the neighbor with highest adjacency
def has_adj_vacant(grid, qubit, adjMat, max_adj):
    M, N = len(grid), len(grid[0])
    maxadj_qubit = None
    for i, j in [(0, -1), (-1, 0), (0, 1), (1, 0)]:
        x, y = qubit
        x += i
        y += j
        if 0 <= x < M and 0 <= y < N:
            if grid[x][y] == '':
                if max_adj == -1 or max_adj < adjMat[x][y]:
                    max_adj = adjMat[x][y]
                    maxadj_qubit = (x, y)
    return maxadj_qubit, max_adj

# Placing Qubits on the Grid

In [91]:
def place_qubits(grid, qubit_priority, adjMat):
    M, N = len(grid), len(grid[0])
    i, j = M//2, N//2
    placed = []

    # Put the highest priority qubit in the middle
    if grid[i][j] == '':
        grid[i][j] = qubit_priority.pop(0)
        placed.append((i, j))
        adjMat = calc_adj(grid)

    while qubit_priority:
        # Find the next cell with highest adjacency adjacent to the already placed qubits
        max_adj = -1
        for x, y in reversed(placed):
            maxadj_qubit, new_max = has_adj_vacant(grid, (x, y), adjMat, max_adj)
            if maxadj_qubit and (max_adj == -1 or max_adj < new_max):
                i, j = maxadj_qubit
                max_adj = new_max
        if max_adj == -1:
            break
        # Place the next qubit in the cell with highest adjacency
        grid[i][j] = qubit_priority.pop(0)
        placed.append((i, j))
        adjMat = calc_adj(grid)
        # print(grid, adjMat, placed, qubit_priority)

    return grid, adjMat

# Path Finding in Grid

In [92]:
# Check if cell(x,y) is in the grid
def is_valid_cell(grid, x, y):
    M, N = len(grid), len(grid[0])
    return 0 <= x < M and 0 <= y < N and grid[x][y] != ''

# Find all possible paths from src to dest using BFS
def find_path_utils(grid, src, dest, visited, path, paths):
    x, y = src
    visited[x][y] = True
    path.append(src)
    if src == dest:
        paths.append(copy.deepcopy(path))
    else:
        for i, j in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            x, y = src
            x += i
            y += j
            if is_valid_cell(grid, x, y) and not visited[x][y]:
                find_path_utils(grid, (x, y), dest, visited, path, paths)
    path.pop()
    visited[x][y] = False

# Find all possible paths from src to dest using BFS
# and return the shortest ones
def find_min_paths(grid, src, dest):
    M, N = len(grid), len(grid[0])
    visited = [[False]*N for _ in range(M)]
    paths = []
    find_path_utils(grid, src, dest, visited, [], paths)

    minpaths, minlen = [], len(paths[0])
    for path in paths:
        if len(path) < minlen:
            minlen = len(path)
    for path in paths:
        if len(path) == minlen:
            minpaths.append(path)
    # print(paths, minpaths)
    return minpaths

# Check if 2 qubits are connected

In [93]:
def find_qubit(grid, qubit):
    M, N = len(grid), len(grid[0])
    for i in range(M):
        for j in range(N):
            if grid[i][j] == qubit:
                return (i, j)
    return None

def check_adjescent(grid, qubit1, qubit2):
    M, N = len(grid), len(grid[0])
    x1, y1 = qubit1
    x2, y2 = qubit2
    if x1 == x2 and abs(y1-y2) == 1:
        return True
    if y1 == y2 and abs(x1-x2) == 1:
        return True
    return False

def interaction_factor(interaction, t, total):
    lookahead_window = 0
    if total != t:
        lookahead_window = math.ceil(math.log(total-t))
    
    elapsed, window_count, occuring = 0, 0, 0
    for i in interaction:
        if i <= t:
            elapsed += 1
        elif window_count <= lookahead_window:
            window_count += 1
            occuring += i
        else: break

    try:
        interval = interaction[-1] - interaction[elapsed]
        return occuring/interval
    except ZeroDivisionError:
        return occuring/interaction[-1]
    except IndexError:
        return 1

# Rearranging qubits on the Grid

In [94]:
def rearrange_qubits(grid, qubit1, qubit2, t, total, interactions):
    M, N = len(grid), len(grid[0])
    if qubit1 == None or qubit2 == None:
        return grid, 0

    freqs = {}
    for path in find_min_paths(grid, qubit1, qubit2):
        freq = 0
        for x,y in path:
            freq += interaction_factor(interactions[grid[x][y]-1], t, total)
        freqs[tuple(path)] = freq
    # print(freqs)

    best_path = []
    if freqs:
        min_freq = min(freqs.values())
        best_path = [path for path, freq in freqs.items() if freq == min_freq][0]
        # print(best_path)

        best_len = len(best_path)
        max_swaps = best_len-2
        gen_combo = lambda s: ((i, s-i) for i in range(s+1))
        ways = list(gen_combo(max_swaps))
        # print(ways)
        
        if len(ways) > 0:
            best_way = random.choice(ways)
            # print(best_way)
        swap_src, swap_dest = best_way

        count = 0
        for i in range(swap_src):
            grid[best_path[i][0]][best_path[i][1]], grid[best_path[i+1][0]][best_path[i+1][1]] = \
                grid[best_path[i+1][0]][best_path[i+1][1]], grid[best_path[i][0]][best_path[i][1]]
            count += 1
        
        j = best_len-1
        for i in range(swap_dest):
            grid[best_path[j][0]][best_path[j][1]], grid[best_path[j-1][0]][best_path[j-1][1]] = \
                grid[best_path[j-1][0]][best_path[j-1][1]], grid[best_path[j][0]][best_path[j][1]]
            j -= 1
            count += 1
        
        count = int(math.ceil(0.8*count))

    return grid, count

### Read the input file and find preference table

In [95]:
# filename = input("Enter the filename: ")
filename = 'rd73_140'
with open(f'benchmarks/revlib_decomposed/{filename}.real', 'r') as f:
    data = f.read()

qubits = int(re.search(r".numvars ([\d]*)", data).group(1))
gates = re.search(r"(?s)(?<=.begin).*?(?=.end)", data, re.MULTILINE).group(0)
lines = gates.strip().split('\n')

# print(qubits, lines)
interaction_times = [[] for _ in range(qubits)]
not_adj_qubits = [[] for _ in range(qubits)]
interacting_qubits = []

for t, line in enumerate(lines, 1):
    params = line.split(' ')
    
    qubit1 = int(re.search(r"\d+", params[1]).group(0))
    interaction_times[qubit1-1].append(t)
    if len(params) == 2: continue
    
    qubit2 = int(re.search(r"\d+", params[2]).group(0))
    interacting_qubits.append((qubit1, qubit2))
    interaction_times[qubit2-1].append(t)

    if abs(qubit1-qubit2) != 1:
        not_adj_qubits[qubit1-1].append(t)
        not_adj_qubits[qubit2-1].append(t)

prefernce = [0]*qubits
for i in range(qubits):
    if sum(interaction_times[i]) != 0:
        prefernce[i] = sum(not_adj_qubits[i])/sum(interaction_times[i])

print(interaction_times)
print(not_adj_qubits)
print(prefernce)

qubit_priority = [i+1 for i in range(qubits)]
qubit_priority.sort(key=lambda x: prefernce[x-1], reverse=True)
print(qubit_priority)

[[2, 3, 5, 6], [1, 3, 4, 5, 6, 13, 14, 16, 17], [8, 9, 11, 12, 14, 15, 16, 17, 29, 30, 32, 33], [19, 20, 22, 24, 25, 27, 28, 30, 31, 32, 33, 45, 46, 48, 49], [35, 36, 38, 40, 41, 43, 44, 46, 47, 48, 49, 61, 62, 64, 65], [51, 52, 54, 56, 57, 59, 60, 62, 63, 64, 65, 72, 73, 75, 76], [67, 68, 70, 71, 73, 74, 75, 76], [1, 2, 4, 7, 9, 10, 11, 12, 13, 15, 23, 25, 26, 27, 28, 29, 31, 39, 41, 42, 43, 44, 45, 47, 55, 57, 58, 59, 60, 61, 63, 71, 72, 74], [7, 8, 10, 18, 20, 21, 22, 23, 24, 26, 34, 36, 37, 38, 39, 40, 42, 50, 52, 53, 54, 55, 56, 58, 66, 68, 69, 70], [18, 19, 21, 34, 35, 37, 50, 51, 53, 66, 67, 69]]
[[2], [1, 4, 13], [8, 9, 11, 12, 15, 29], [19, 20, 22, 24, 25, 27, 28, 31, 45], [35, 36, 38, 40, 41, 43, 44, 47, 61], [51, 52, 54, 56, 57, 59, 60, 63, 72], [67, 68, 70], [1, 2, 4, 9, 11, 12, 13, 15, 25, 27, 28, 29, 31, 41, 43, 44, 45, 47, 57, 59, 60, 61, 63, 72], [8, 20, 22, 24, 36, 38, 40, 52, 54, 56, 68, 70], [19, 35, 51, 67]]
[0.125, 0.22784810126582278, 0.37168141592920356, 0.503131

# Qubit Placement and Swap Counts

In [96]:
# M, N = map(int, input("Enter the grid size: ").split())
M, N = 4, 3
grid = [['']*N for _ in range(M)]
adjMat = calc_adj(grid)
grid, adjMat = place_qubits(grid, qubit_priority, adjMat)
print(grid, adjMat)

best_grid, min_swap = [], 1000
for i in range(10000):
    temp_grid = copy.deepcopy(grid)
    swap_count = 0
    for t, interaction in enumerate(interacting_qubits, 1):
        if len(interaction) == 2:
            qubit1, qubit2 = interaction
            x1, y1 = find_qubit(temp_grid, qubit1)
            x2, y2 = find_qubit(temp_grid, qubit2)

            if not check_adjescent(temp_grid, (x1, y1), (x2, y2)):
                # print(f"\nswap at line {t}: {interaction}")
                temp_grid, swap = rearrange_qubits(temp_grid, (x1, y1), (x2, y2), t, len(lines), interaction_times)
                swap_count += swap
                # print(grid)

    # print("\nGrid: ",grid)
    # print(f"swap count: {swap_count}")
    if swap_count < min_swap:
        min_swap = swap_count
        best_grid = grid
        print(f"min swap count: {min_swap} at iteration {i}")

print("\nBest Grid: ", best_grid)
print(f"min swap count: {min_swap}")

[['', 4, ''], [5, 6, 9], [1, 8, 2], [7, 3, 10]] [[0, -1, 0], [-1, -1, -1], [-1, -1, -1], [-1, -1, -1]]
min swap count: 83 at iteration 0
min swap count: 78 at iteration 1
min swap count: 59 at iteration 2
min swap count: 57 at iteration 8
min swap count: 49 at iteration 25
min swap count: 47 at iteration 41
min swap count: 41 at iteration 57
min swap count: 40 at iteration 266
min swap count: 39 at iteration 1311
min swap count: 38 at iteration 7758

Best Grid:  [['', 4, ''], [5, 6, 9], [1, 8, 2], [7, 3, 10]]
min swap count: 38
